In [62]:
from sklearn.datasets import make_classification
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import numpy as np

C:\Program Files\Anaconda2\envs\python3_5\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# We use a utility to generate artificial classification data.
X, y = make_classification(n_samples=100, n_informative=10, n_classes=3)
sss = StratifiedShuffleSplit(y, n_iter=1, test_size=0.5, random_state=0)
for train_idx, test_idx in sss:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))    

In [132]:
def calculate_AER(S, P, A):
    s_a, p_a, len_s, len_a = 0, 0, 0, 0
    for s, p, a in zip(S, P, A):
        s_a += len(list(set(s).intersection(a)))
        p_a += len(list(set(p).intersection(a)))
        len_s += len(s)
        len_a += len(a)
    print ("s_a", s_a)
    p_a += s_a
    print ("p_a", p_a)
    aer = (s_a + p_a) / (len_s + len_a)
    print ("aer", 1.-aer)
    
    return 1. - aer 


def calculate_one_AER(S, P, A):
    
    s_a = len(list(set(S).intersection(A)))
    print ("s_a", s_a)
    p_a = len(list(set(P).intersection(A))) + s_a
    print ("p_a", p_a)
    aer = (s_a + p_a) / (len(S) + len(A))
    print ("aer", 1.-aer)
    
    return 1. - aer 
    
def write_file(strs, file_name):
    alignment_test = open(file_name,"w", encoding='utf8') 
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

In [16]:
S = [['3-2', '1-1']]
P = [['2-2']]
A = [['1-2', '1-1', '3-4']]
calculate_AER(S, P, A)

['3-2', '1-1'] ['2-2'] ['1-2', '1-1', '3-4']
s_a 1
p_a 1
aer 0.6


0.6

In [17]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open

docXml = './celex_21990A0714.m.wa'
root = minidom.parse(docXml).getElementsByTagName("sentences")

english_sentences = []
czech_sentences = []
sure = []
possible = []

for sent in root[0].childNodes :
    
    #on retire les noeuds vides
    if sent.localName == None : continue

    sentID = sent.getAttribute('id')

    #pour chaque noeud de phrase on parcourt les tokens de la phrase, on regarde leur longueur et on calcule
    #le numero du premier charactère ... sera utile pour la suite pour avoir l'ID html du charactère correspondant
    tokenNodes = sent.childNodes
    
    english_sentences.append(tokenNodes[1].childNodes[0].data)
    czech_sentences.append(tokenNodes[3].childNodes[0].data)
    sure.append(tokenNodes[5].childNodes[0].data)
    try:
        possible.append(tokenNodes[7].childNodes[0].data)
    except Exception: 
        possible.append('')

        
write_file(english_sentences, "alignment_test.en")
write_file(czech_sentences, "alignment_test.cz")

In [19]:
result = open("corpus.en2cz.gdfa.ali", "r")
aer = 0.
S, P, A = [], [], []

for line, s_, p_ in zip(result, sure, possible):
    a = line.strip().split(" ")
    for number, i in zip(a, range(len(a))):
        n1 = int(number.split("-")[0]) + 1 
        n2 = int(number.split("-")[1]) + 1 
        number = str(n1) + "-" + str(n2)
        a[i] = number
    S.append(s_.strip().split(" "))
    P.append(p_.strip().split(" "))
    A.append(a)
    
aer = calculate_AER(S, P, A)

print(aer)
result.close()

['1-1', '3-2', '3-3', '3-4', '3-5', '4-6', '5-7', '5-8', '5-9', '5-10', '8-12'] ['2-3', '6-11', '7-12'] ['1-1', '2-2', '3-3', '4-4', '5-5', '6-6', '7-7', '8-8']
['2-1', '5-4', '7-2', '8-3', '9-5', '10-5', '11-6', '12-7', '14-8', '14-9', '14-10', '14-11', '15-15', '19-18', '21-19', '21-20', '21-21', '21-22', '21-23', '21-24', '22-25', '25-26', '26-27', '28-28', '29-29', '30-30', '31-31', '32-32', '33-33', '34-34', '35-35', '37-36', '39-39', '40-37', '41-40'] ['1-1', '3-4', '4-4', '6-4', '13-9', '15-12', '15-13', '15-14', '16-16', '16-17', '18-18', '20-20', '24-28', '36-36', '38-37', '39-38'] ['1-1', '2-2', '3-3', '4-4', '5-5', '6-6', '7-7', '8-8', '9-9', '10-10', '11-11', '12-12', '13-13', '14-14', '15-15', '16-16', '17-17', '18-18', '19-19', '20-20', '21-21', '22-22', '23-23', '24-24', '25-25', '26-26', '27-27', '27-29', '28-28', '29-27', '30-32', '31-32', '32-30', '32-31', '32-32', '33-33', '34-34', '35-35', '36-36', '37-37', '38-38', '39-39', '40-40', '41-41']
['2-3', '3-1', '4-4', '

# Preprocessing training file: En-Cz

In [7]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open
import glob

from lxml import html
from lxml import etree
# tree = html.parse(path)

def write_file(strs, file_name):
    alignment_test = open(file_name,"w", encoding='utf8') 
    print("Writing file: ", file_name)
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

english_sentences = []
czech_sentences = []
sure = []
possible = []

file = "E:/Working/Intership2017/en-cz/raw/training/news-commentary-v11.cs-en.xliff"
print("Processing file:", file)
parser = html.parse(file)
sentences = parser.getroot().getchildren()[0].getchildren()[0].getchildren()[0].getchildren()
for s in sentences:
    child = s.getchildren()
    values = []
    for c in child:
        if (c.text != None):
            values.append(str(c.text))
        else:
            values.append('')
    english_sentences.append(values[1])
    czech_sentences.append(values[0])

assert len(english_sentences)==len(czech_sentences)
write_file(english_sentences, "training.en-cz.en")
write_file(czech_sentences, "training.en-cz.cz")
print("Done !")

Processing file: E:/Working/Intership2017/en-cz/raw/training/news-commentary-v11.cs-en.xliff
Writing file:  training.en-cz.en
Writing file:  training.en-cz.cz
Done !


# Preprocessing testing file: En-Cz

In [143]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open
import glob

from lxml import html
from lxml import etree
# tree = html.parse(path)

def write_file(strs, file_name):
    alignment_test = open(file_name, "w", encoding='utf8') 
    print("Writing file: ", file_name)
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

english_sentences = []
czech_sentences = []
sure = []
possible = []

files = glob.glob("E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0/*.wa")
for file in files:
    print("Processing file:", file)
    parser = html.parse(file)
    root = parser.getiterator("sentences")
    for r in root:
        sentences = r.getchildren()
        for s in sentences:
            child = s.getchildren()
            values = []
            for c in child:
                if (c.text != None):
                    values.append(str(c.text))
                else:
                    values.append('')
            english_sentences.append(values[0])
            czech_sentences.append(values[1])
            sure.append(values[2])
            possible.append(values[3])
    
    assert len(english_sentences)==len(czech_sentences)==len(sure)==len(possible)

write_file(english_sentences, "testing.en-cz.en")
write_file(czech_sentences, "testing.en-cz.cz")
align_test = []
for (s, p) in zip(sure, possible):
    align_test.append(s)
    align_test.append(p)
write_file(align_test, "testing.en-cz.aligment")
print("Done !")

Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\books_three_men_in_boat.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_21990A0714.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_22000A0411.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_31958L0003.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_31962L2645.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_31965L0079.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_31970L0220.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_31975L0033.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_31980L0799.wa
Processing file: E:/Working/Intership2017/en-cz/raw/testing/aligned_cz-en_1.0\celex_31985L0203.wa
Processing fi

# Preprocessing training file: En-Fr

In [16]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open
import glob
import os

from lxml import html
from lxml import etree

def write_file(strs, file_name):
    alignment_test = open(file_name, "w", encoding='utf8') 
    print("Writing file: ", file_name)
    for s in strs:
        alignment_test.write(s + "\n") 
    alignment_test.close()

def write_file_list(file_list, file_name):
    with open(file_name, 'w') as outfile:
        for fname in file_list:
            print("Processing file:", os.path.basename(fname))
            with open(fname) as infile:
                outfile.write(infile.read())
                outfile.write("\n")

e_house_files = glob.glob("E:/Working/Intership2017/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/*.e")
f_house_files = glob.glob("E:/Working/Intership2017/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/house/debates/development/training/*.f")
write_file_list(e_house_files, "E:/Working/Intership2017/en-fr/formatted/training/training.house.en-fr.en")
write_file_list(f_house_files, "E:/Working/Intership2017/en-fr/formatted/training/training.house.en-fr.fr")

e_senate_files = glob.glob("E:/Working/Intership2017/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/senate/debates/development/training/*.e")
f_senate_files = glob.glob("E:/Working/Intership2017/en-fr/raw/training/hansard.36/Release-2001.1a/sentence-pairs/senate/debates/development/training/*.f")
write_file_list(e_senate_files, "E:/Working/Intership2017/en-fr/formatted/training/training.senate.en-fr.en")
write_file_list(f_senate_files, "E:/Working/Intership2017/en-fr/formatted/training/training.senate.en-fr.fr")
print("Done !")

Processing file: hansard.36.1.house.debates.001.e
Processing file: hansard.36.1.house.debates.002.e
Processing file: hansard.36.1.house.debates.003.e
Processing file: hansard.36.1.house.debates.004.e
Processing file: hansard.36.1.house.debates.005.e
Processing file: hansard.36.1.house.debates.006.e
Processing file: hansard.36.1.house.debates.007.e
Processing file: hansard.36.1.house.debates.008.e
Processing file: hansard.36.1.house.debates.009.e
Processing file: hansard.36.1.house.debates.010.e
Processing file: hansard.36.1.house.debates.011.e
Processing file: hansard.36.1.house.debates.012.e
Processing file: hansard.36.1.house.debates.013.e
Processing file: hansard.36.1.house.debates.014.e
Processing file: hansard.36.1.house.debates.015.e
Processing file: hansard.36.1.house.debates.016.e
Processing file: hansard.36.1.house.debates.017.e
Processing file: hansard.36.1.house.debates.018.e
Processing file: hansard.36.1.house.debates.019.e
Processing file: hansard.36.1.house.debates.020.e
